In [2]:
# Import relevant modules
# ImagePrediction is a helper module (helpers/image_models) that assists with setting up the image recognition models
# and getting a prediction.
from helpers.image_models import ImagePrediction
# Ground truth helper so we can see the ground truth of an Imagenet image.
from validate_ground_truth.imagenet_ground_truth import ImagenetGroundTruth
# Instantiates the ImagePrediction object with the vgg16 model
predictor = ImagePrediction(model_name="vgg16")
# Instantiates the ImagenetGroundTruth object
ground_truth = ImagenetGroundTruth()
from PIL import Image

2024-03-08 12:49:03.695347: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 12:49:03.695385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 12:49:03.696558: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-08 12:49:03.703184: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 12:49:04.450924: W tensorflow/compiler/tf2

1 Physical GPUs, 1 Logical GPUs
Version:  2.15.0
Eager mode:  True
GPU is available


In [13]:
# Gets the fitness prediction for the arctic fox image.
arctic_fox_loc = "./initial_test_set/arctic-fox.jpg"
arctic_fox = predictor.get_prediction(arctic_fox_loc)
print(arctic_fox)

[('n02120079', 'Arctic_fox', 0.9021174), ('n02134084', 'ice_bear', 0.0374203), ('n02114548', 'white_wolf', 0.026164168), ('n02441942', 'weasel', 0.025217617), ('n02326432', 'hare', 0.0026364862), ('n02111889', 'Samoyed', 0.0025606712), ('n02109961', 'Eskimo_dog', 0.0014539941), ('n02110185', 'Siberian_husky', 0.0006840576), ('n02442845', 'mink', 0.00034034965), ('n02114367', 'timber_wolf', 0.0002977582)]


In [14]:
# Create an image object from the arctic-fox jpg
arctic_fox = Image.open(arctic_fox_loc)
# Flip the image so it's upside down
af_flipped = arctic_fox.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)
# Save the image
output_flipped = "output/af_flipped.jpg" 
af_flipped.save(output_flipped)

In [15]:
# Show us the prediction for the flipped image .. interesting outcome
print(predictor.get_prediction(output_flipped))

[('n02134084', 'ice_bear', 0.19055848), ('n03888257', 'parachute', 0.17134756), ('n02120079', 'Arctic_fox', 0.044043142), ('n01796340', 'ptarmigan', 0.03144516), ('n02441942', 'weasel', 0.017049879), ('n03958227', 'plastic_bag', 0.016838005), ('n03188531', 'diaper', 0.015459845), ('n02058221', 'albatross', 0.013839627), ('n03743016', 'megalith', 0.011456719), ('n03344393', 'fireboat', 0.0114172725)]


In [16]:
# Create an image object from the arctic-fox jpg
arctic_fox = Image.open(arctic_fox_loc)
# Flip the image so it's upside down
af_flipped = arctic_fox.effect_spread(25)
# Save the image
output_spread = "output/af_spread.jpg"
af_flipped.save(output_spread)

In [17]:
# Show us the prediction for the flipped image .. interesting outcome
print(predictor.get_prediction(output_spread))

[('n03888257', 'parachute', 0.14076778), ('n09288635', 'geyser', 0.08659609), ('n02120079', 'Arctic_fox', 0.055041064), ('n02441942', 'weasel', 0.04239273), ('n03743016', 'megalith', 0.035429187), ('n04228054', 'ski', 0.028895907), ('n01796340', 'ptarmigan', 0.024988918), ('n02134084', 'ice_bear', 0.023643998), ('n03344393', 'fireboat', 0.020294352), ('n03388043', 'fountain', 0.01771445)]


In [92]:
import tensorflow as tf
import os
import pathlib
from helpers.image_models import decode_predictions
from pprint import pprint

dataset_path = pathlib.Path(os.getcwd() + "/initial_test_set")
image_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path.as_posix(), 
    image_size=(224, 224), 
    batch_size=32,
    shuffle=False
)

# get predictions with filenames
predictions = predictor.model.predict(image_dataset)

decoded = decode_predictions(predictions, top=1)

for i, image in enumerate(image_dataset.file_paths):
    print(f"Filename: {image.split('/')[-1]}")
    print(f"Prediction: {decoded[i]}")


Found 10 files belonging to 1 classes.
1/1 [==============================] - 0s 289ms/step
Filename: arctic-fox.jpg
Prediction: [('n02120079', 'Arctic_fox', 0.9508856)]
Filename: castle.jpg
Prediction: [('n02980441', 'castle', 0.97413325)]
Filename: iguana.jpg
Prediction: [('n01677366', 'common_iguana', 0.97907984)]
Filename: llama.jpg
Prediction: [('n02437616', 'llama', 0.96955395)]
Filename: salmon.jpg
Prediction: [('n02536864', 'coho', 0.87779164)]
Filename: siamese_cat.jpg
Prediction: [('n02123597', 'Siamese_cat', 0.9406736)]
Filename: tennis_racket.jpg
Prediction: [('n04039381', 'racket', 0.88523024)]
Filename: tiger.jpg
Prediction: [('n02129604', 'tiger', 0.9228465)]
Filename: truck.jpg
Prediction: [('n04467665', 'trailer_truck', 0.83496106)]
Filename: wolf.jpg
Prediction: [('n02114367', 'timber_wolf', 0.8401957)]


In [93]:
# 10 transformation functions
# 1. flip
def flip_image(image):
    return image.transpose(method=Image.Transpose.FLIP_TOP_BOTTOM)
# 2. rotate
def rotate_image(image, degrees):
    return image.rotate(degrees)
# 3. zoom
def zoom_image(image, zoom_factor):
    return image.resize((int(image.width * zoom_factor), int(image.height * zoom_factor)))
# 4. brightness
def brightness_image(image, factor):
    return image.point(lambda p: p * factor)
# 5. contrast
def contrast_image(image, factor):
    return image.point(lambda p: p * factor)
# 6. saturation
def saturation_image(image, factor):
    return image.point(lambda p: p * factor)
# 7. hue
def hue_image(image, factor):
    return image.point(lambda p: p * factor)
# 8. shear
def shear_image(image, factor):
    return image.transform(image.size, Image.AFFINE, (1, factor, 0, 0, 1, 0))
# 9. translate
def translate_image(image, x, y):
    return image.transform(image.size, Image.AFFINE, (1, 0, x, 0, 1, y))
# 10. spread
def spread_image(image, factor):
    return image.effect_spread(factor)

In [ ]:
# automatically generate transformed images for each image in the dataset and save them to a new directory
def generate_transformed_images(dataset_path, output_path, transformations):
    for subdir, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            image = Image.open(file_path)
            for transform in transformations:
                transformed_image = transform(image)
                output_file_path = os.path.join(output_path, f"{file.split('.')[0]}_{transform.__name__}.jpg")
                transformed_image.save(output_file_path)

generate_transformed_images(dataset_path, "output_set/test", [flip_image, rotate_image, zoom_image, brightness_image, contrast_image, saturation_image, hue_image, shear_image, translate_image, spread_image])

In [ ]:
transformed_dataset_path = pathlib.Path(os.getcwd() + "/output_set")
transformed_image_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    transformed_dataset_path.as_posix(), 
    image_size=(224, 224), 
    batch_size=32,
    shuffle=False
)

# create a new dataset from the transformed images
predictions = predictor.model.predict(transformed_image_dataset)

decoded = decode_predictions(predictions, top=1)

for i, image in enumerate(transformed_image_dataset.file_paths):
    print(f"Filename: {image.split('/')[-1]}")
    print(f"Prediction: {decoded[i]}")